In [3]:
import os
import json
import time
import azureml
import logging
from azureml.core import Workspace, Run, Datastore, Experiment

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.21


In [5]:
# use this code to set up config file
#subscription_id ='<SUB_ID>'
#resource_group ='<RESOURCE_GROUP>'
#workspace_name = '<WORKSPACE>'

#try:
#    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
#    ws.write_config()
#    print('Workspace configuration succeeded. You are all set!')
#except:
#    print('Workspace not found. TOO MANY ISSUES!!!')

ws = Workspace.from_config()

Found the config file in: C:\projects\FoodAI\aml_config\config.json


# Compute Environment

In [6]:
cluster = 'sauron'
try:
    compute = ComputeTarget(workspace=ws, name=cluster)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', min_nodes=1, max_nodes=6)
    compute = ComputeTarget.create(ws, cluster, compute_config)
    compute.wait_for_completion(show_output=True)

Found existing compute target


# Manage Data

In [15]:
# Register datastore
#ds = Datastore.register_azure_blob_container(workspace=ws,
#                                             datastore_name='foodai',
#                                             container_name='foodai',
#                                             account_name='robotdatasets',
#                                             account_key='<KEY>',
#                                             create_if_not_exists=False)
for name, ds in ws.datastores.items():
    print(name, ds.datastore_type)

workspaceblobstore AzureBlob
workspacefilestore AzureFile
images_datastore AzureBlob
coco AzureBlob
robotdatasets__coco AzureBlob
foodai AzureBlob


In [16]:
ds = Datastore.get(ws, datastore_name='foodai')

# Run Experiment

In [63]:
from azureml.train.dnn import PyTorch
from azureml.widgets import RunDetails

script_params = {
    "--data": ds.as_mount(), # don't download
    "--output": "outputs",
    "--epochs": 25,
    "--batch": 8,
    "--lr": 0.001,
}

# Create and run experiment
foodEstimator = PyTorch(source_directory='./',
                        script_params=script_params,
                        compute_target=compute,
                        entry_script='train.py',
                        conda_packages=['matplotlib'],
                        use_gpu=True)

In [64]:
exp = Experiment(ws, 'food_ai')
run = exp.submit(foodEstimator)
run

Experiment,Id,Type,Status,Details Page,Docs Page
food_ai,food_ai_1554703972_f1641a9b,azureml.scriptrun,Queued,Link to Azure Portal,Link to Documentation


In [65]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [109]:
import requests
import numpy as np
from PIL import Image
from io import BytesIO
import onnxruntime as rt
from torchvision import transforms

sess = rt.InferenceSession("model.onnx")
input_name = sess.get_inputs()[0].name
classes = ['hot_dog', 'pizza']
t = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

In [119]:
def score(img):
    response = requests.get(img)
    img = Image.open(BytesIO(response.content))
    v = t(img)
    pred_onnx = sess.run(None, {input_name: v.unsqueeze(0).numpy()})
    return classes[np.argmax(pred_onnx)]

In [120]:
pizza = 'https://images-gmi-pmc.edge-generalmills.com/f4c0a86f-b080-45cd-a8a7-06b63cdb4671.jpg'
score(pizza)

'pizza'

In [121]:
hot_dog = 'https://leitesculinaria.com/wp-content/uploads/fly-images/96169/best-hot-dog-recipe-fi-400x225-c.jpg'
score(hot_dog)

'hot_dog'